# FBI background checks - monthly


### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import datetime as dt
import numpy as np
import altair as alt
import altair_stiles as altstiles

/Users/stiles/.local/share/virtualenvs/grid-notebooks-Amv9DX-2/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [5]:
month_year_updated = dt.date.today().strftime("%m_%Y")

---

### Bless you @jsvine

In [6]:
url = "https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv"

### Read data

In [7]:
src = pd.read_csv(url)

In [8]:
src.iloc[0]

month                        2022-04
state                        Alabama
permit                       16150.0
permit_recheck                 236.0
handgun                      20593.0
long_gun                     12513.0
other                         1249.0
multiple                        1041
admin                            0.0
prepawn_handgun                 21.0
prepawn_long_gun                 2.0
prepawn_other                    0.0
redemption_handgun            2214.0
redemption_long_gun            852.0
redemption_other                 8.0
returned_handgun                37.0
returned_long_gun                0.0
returned_other                   0.0
rentals_handgun                  0.0
rentals_long_gun                 0.0
private_sale_handgun            16.0
private_sale_long_gun           19.0
private_sale_other               4.0
return_to_seller_handgun         0.0
return_to_seller_long_gun        1.0
return_to_seller_other           0.0
totals                         54956
N

In [9]:
src["year"] = src["month"].str[:4]

---

### AP states

In [10]:
ap_states = {
    "Alabama": "Ala.",
    "Alaska": "Alaska",
    "Arizona": "Ariz.",
    "Arkansas": "Ark.",
    "California": "Calif.",
    "Colorado": "Colo.",
    "Connecticut": "Conn.",
    "Delaware": "Del.",
    "Florida": "Fla.",
    "Georgia": "Ga.",
    "Hawaii": "Hawaii",
    "Idaho": "Idaho",
    "Illinois": "Ill.",
    "Indiana": "Ind.",
    "Iowa": "Iowa",
    "Kansas": "Kan.",
    "Kentucky": "Ky.",
    "Louisiana": "La.",
    "Maine": "Md.",
    "Maryland": "Mass.",
    "Massachusetts": "Maine",
    "Michigan": "Mich.",
    "Minnesota": "Minn.",
    "Mississippi": "Miss.",
    "Missouri": "Mo.",
    "Montana": "Mont.",
    "Nebraska": "Neb.",
    "Nevada": "Nev.",
    "New Hampshire": "N.H.",
    "New Jersey": "N.J.",
    "New Mexico": "N.M.",
    "New York": "N.Y.",
    "North Carolina": "N.C.",
    "North Dakota": "N.D.",
    "Ohio": "Ohio",
    "Oklahoma": "Okla.",
    "Oregon": "Ore.",
    "Pennsylvania": "Pa.",
    "Rhode Island": "R.I.",
    "South Carolina": "S.C.",
    "South Dakota": "S.D.",
    "Tennessee": "Tenn.",
    "Texas": "Texas",
    "Utah": "Utah",
    "Vermont": "Vt.",
    "Virginia": "Va.",
    "Washington": "Wash.",
    "West Virginia": "W.Va.",
    "Wisconsin": "Wis.",
    "Wyoming": "Wyo",
}

In [11]:
src["ap_state"] = src["state"].map(ap_states)

### Exclude permits figures

In [12]:
src["totals_no_permits"] = src[["handgun", "multiple", "long_gun"]].sum(axis=1)

In [13]:
src["year"] = src["month"].str[:4]

In [14]:
src["month"] = src["month"].str[-2:]

In [15]:
monthnames = {
    "01": "Jan.",
    "02": "Feb.",
    "03": "Mar.",
    "04": "Apr.",
    "05": "May.",
    "06": "Jun.",
    "07": "Jul.",
    "08": "Aug.",
    "09": "Sep.",
    "10": "Oct.",
    "11": "Nov.",
    "12": "Dec.",
}

In [16]:
src["month_name"] = src["month"].map(monthnames)

In [17]:
src.head()

,month,state,permit,permit_recheck,handgun,long_gun,other,multiple,admin,prepawn_handgun,prepawn_long_gun,prepawn_other,redemption_handgun,redemption_long_gun,redemption_other,returned_handgun,returned_long_gun,returned_other,rentals_handgun,rentals_long_gun,private_sale_handgun,private_sale_long_gun,private_sale_other,return_to_seller_handgun,return_to_seller_long_gun,return_to_seller_other,totals,year,ap_state,totals_no_permits,month_name
0,04,Alabama,16150.0,236.0,20593.0,12513.0,1249.0,1041,0.0,21.0,2.0,0.0,2214.0,852.0,8.0,37.0,0.0,0.0,0.0,0.0,16.0,19.0,4.0,0.0,1.0,0.0,54956,2022,Ala.,34147.0,Apr.
1,04,Alaska,354.0,19.0,3425.0,2946.0,391.0,192,0.0,2.0,1.0,0.0,114.0,81.0,1.0,39.0,14.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,7589,2022,Alaska,6563.0,Apr.
2,04,Arizona,25746.0,1305.0,20628.0,9300.0,1814.0,1253,0.0,7.0,5.0,2.0,1211.0,407.0,5.0,172.0,18.0,0.0,0.0,0.0,16.0,10.0,0.0,0.0,0.0,0.0,61899,2022,Ariz.,31181.0,Apr.
3,04,Arkansas,2043.0,364.0,7659.0,5376.0,483.0,390,4.0,3.0,8.0,0.0,1037.0,832.0,2.0,0.0,0.0,0.0,0.0,0.0,12.0,7.0,2.0,0.0,0.0,0.0,18222,2022,Ark.,13425.0,Apr.
4,04,California,17944.0,12972.0,38217.0,25572.0,5595.0,5,1.0,1.0,2.0,0.0,807.0,393.0,33.0,1986.0,1117.0,158.0,0.0,0.0,7491.0,3186.0,698.0,23.0,7.0,0.0,116208,2022,Calif.,63794.0,Apr.


In [18]:
df = src[
    [
        "month",
        "month_name",
        "year",
        "state",
        "ap_state",
        "handgun",
        "multiple",
        "long_gun",
        "totals_no_permits",
    ]
]

In [19]:
df.columns

Index(['month', 'month_name', 'year', 'state', 'ap_state', 'handgun',
       'multiple', 'long_gun', 'totals_no_permits'],
      dtype='object')

In [20]:
grouped = (
    df[df["year"] > "2008"]
    .groupby(["year", "month", "month_name"])
    .agg(
        {
            "handgun": sum,
            "long_gun": sum,
            "multiple": sum,
            "totals_no_permits": sum,
        }
    )
    .reset_index()
)
grouped.head()

,year,month,month_name,handgun,long_gun,multiple,totals_no_permits
0,2009,01,Jan.,355173.0,415474.0,24035,794682.0
1,2009,02,Feb.,394590.0,427636.0,26462,848688.0
2,2009,03,Mar.,405784.0,469117.0,26724,901625.0
3,2009,04,Apr.,352352.0,400232.0,24723,777307.0
4,2009,05,May.,282826.0,314167.0,24031,621024.0


In [21]:
grouped.sort_values("totals_no_permits", ascending=False).head()

,year,month,month_name,handgun,long_gun,multiple,totals_no_permits
134,2020,03,Mar.,1392677.0,758073.0,57328,2208078.0
47,2012,12,Dec.,876673.0,1224465.0,38438,2139576.0
83,2015,12,Dec.,1201747.0,873335.0,34059,2109141.0
137,2020,06,Jun.,1371811.0,625791.0,39977,2037579.0
144,2021,01,Jan.,1124621.0,713492.0,38138,1876251.0


In [23]:
heatmap = (
    (
        alt.Chart(grouped)
        .mark_rect()
        .encode(
            x=alt.X(
                "year:O",
                axis=alt.Axis(title=""),
            ),
            y=alt.Y(
                "month:O",
                axis=alt.Axis(title=""),
            ),
            color=alt.Color(
                ("totals_no_permits:Q"),
                title="",
                scale=alt.Scale(scheme="teals", reverse=False),
            ),
        )
    )
    .properties(
        width=380,
        height=300,
        title=f" ",
    )
    .configure_legend(orient="top", tickCount=1)
)
heatmap.display()

alt.Chart(...)

In [24]:
grouped.to_csv("data/processed/checks-grouped.csv", index=False)

In [25]:
grouped.pivot_table(
    index="month_name", values="totals_no_permits", columns="year"
).reset_index().to_csv("data/processed/checks-grouped.csv", index=False)